# QUBO Representation of Combined Knapsack and Set Cover Problem

CDL Quantum Hackathon 2021  
Team ZebraKet   
Ziwei Qiu (ziweiqiu@g.harvard.edu), Alex Khan, Theo Cleland, Ehsan Torabizadeh

## Problem Definition
As a grocery store manager, you want to re-stock. This notebook provides a solution to help you make decisions what items to buy to maximize your profit and where to buy them to minimize the number of suppliers you need to purchase from. 

Let's first define the problem, clarify the parameters and variables. Parameters are denoted by uppercase letters and variables by lowercase letters.

There are $N$ suppliers in the market and $M$ items in total in your inventory. $i$ is the supplier index, so $i=1,2,...,N$. $\alpha$ is the item index, so $\alpha=1,2,...,M$.The cost of an item can differ between suppliers, so $W_{\alpha,i}$ denotes the cost of item $\alpha$ in supplier $i$. $V_{\alpha}$ denotes the selling price of item $\alpha$ in your grocery store to your customers. $W$ is the budget you have for purchase.

There are four variables in this problem:  
(1) $x_i$ is a binary variable which equals to 1 if the supplier $i$ is chosen.   
(2) $y_{\alpha,m}$ is a binary variable which equals to 1 if among the suppliers you choose, there are $m$ of suppliers have the item $\alpha$ available.   
(3) $z_{\alpha,i}$ is a discrete variable which denotes the quantity of item $\alpha$ you decide to purchase from supplier $i$.   
(4) $w_n$ is a binary variable which equals to 1 if the total cost of your purchase is equal to $n$.

Your decisions will be based on these variable values. In terms of cash flow, your negative cash flow will be $\sum_{\alpha, i} W_{\alpha,i} z_{\alpha,i}$ by purchasing from. Your potential positive cash flow will be $\sum_{\alpha, i} V_{\alpha} z_{\alpha,i}$ from your customers.

Here are the considerations:  
(1) You may build long-term collaboration with certain suppliers. To reduce the cost, you want to minimize the number of suppliers you will work with. The chosen suppliers should cover all the items in your inventory. This is a set cover problem.  
(2) Based on current item costs $W_{\alpha,i}$ from suppliers and your selling price $V_{\alpha}$, you want to decide what items and how many of them to purchase this time. Your negative cash flow needs to be within the budget $W$, while you want to maximize the potential positive cash flow. This is a Knapsack problem.  
(3) In the real world, considerations (1) and (2) can compete. It is likely that minimizing the number of suppliers doensn't optimize your profit, while maximizing your profit may require you to work with more suppliers which can add overhead expenses (negoitations, travelling, etc.). We provide a solution to deal with this dilemma. We represent this combined optimization problem with QUBO formulation and solving it by running on D-Wave's quantum hardware.

## QUBO Representation

We define the following Hamiltonian to represent our problem. 

$H_1 = A_1\sum_{\alpha=1}^{M}\left(1-\sum_{m=1}^{N}y_{\alpha,m}\right)^2 + A_1\sum_{\alpha=1}^{M}\left(\sum_{m=1}^{N}my_{\alpha,m}-\sum_{i:\alpha\in V_i}x_i \right)^2 + 
B_1\sum_{i=1}^{N} x_i$

$H_2=A_2\left(1 - \sum_{n=1}^{W} w_{n}\right)^{2} + A_2' \left(\sum_{n=1}^{W}n w_{n} - \sum_{\alpha=1}^{M}\sum_{i=1}^{N} W_{\alpha,i} z_{\alpha,i}\right)^{2}  - B_2\sum_{\alpha=1}^{M}\sum_{i=1}^{N}V_{\alpha}z_{\alpha,i}$ 

$H_3=C \sum_{\alpha=1}^{M}\sum_{i=1}^{N} z_{\alpha,i}\left(1-x_i\right)$

In [5]:
from dimod import BinaryQuadraticModel
from dimod import DiscreteQuadraticModel
from dimod import ExactSolver
from neal import SimulatedAnnealingSampler
from itertools import combinations
from dwave.system import LeapHybridSampler
from dwave.system import LeapHybridDQMSampler
import os
import numpy as np
import pandas as pd